In [11]:
import pandas as pd
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

from numpy import hstack, vstack
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings(action='ignore')

In [12]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()

'c:\\Users\\Ansh\\Desktop\\MEST\\HMG3\\Revision\\Rapid_fin\\deep learning modeling'

In [13]:
csv_add = find_directory(foldername = [], filename = 'SOC_Point_Data.csv')
dat = pd.read_csv(csv_add, index_col = (0,1,2,3,4))

In [14]:
def Get_Data(dat) :
    
    RPT_MODE = "0.1C"
    SOC_Range = [9,10,11,12]

    Time_Range = range(6, 15, 2)
    SOC_Range = [str(i) for i in SOC_Range]
    

    Data = dat

    X = Data.loc[RPT_MODE, "0" : "16"]
    Y = Data.loc[RPT_MODE, ["SOH", "Next_SOH", "Ratio_SOH", "Ratio_CYC"]].groupby(level = ["Next", "Path", "Number"]).mean()
    
    y = pd.Series(Y["Next_SOH"] - Y["SOH"], name = "Delta_SOH")
    
    Y = pd.concat([Y, y], axis = 1)

    X_seek = X.loc[X.index.get_level_values("Time").isin(Time_Range), SOC_Range]


    X_std = X_seek.groupby(level = ["Next", "Path", "Number"]).std()
    
    return X_std, Y

In [15]:
def Even_Split(X, y, test_size, rs) :

    X_M = X.xs(key = 'M', level = 'Next', drop_level = False)
    X_D = X.xs(key = 'D', level = 'Next', drop_level = False)
    X_H = X.xs(key = 'H', level = 'Next', drop_level = False)
    
    XX = {"M" : X_M, "D" : X_D, "H" : X_H}
    
    y_M = y.xs(key = 'M', level = 'Next', drop_level = False)
    y_D = y.xs(key = 'D', level = 'Next', drop_level = False)
    y_H = y.xs(key = 'H', level = 'Next', drop_level = False)
    
    yy = {"M" : y_M, "D" : y_D, "H" : y_H}
    
    
    XXX = {"M" : [], "D" : [], "H" : []}
    
    yyy = {"M" : [], "D" : [], "H" : []}
    
    
    for n in ["M", "D", "H"] :
        for path in range(1,5) :
            X_path = XX[n].loc[XX[n].index.get_level_values(level = 'Path').str.len() == path]
            y_path = yy[n].loc[yy[n].index.get_level_values(level = 'Path').str.len() == path]
            
            XXX[n].append(X_path)
            yyy[n].append(y_path)
            
            
    XX_tn = {"M" : [], "D" : [], "H" : []}
    XX_te = {"M" : [], "D" : [], "H" : []}
    
    yy_tn = {"M" : [], "D" : [], "H" : []}
    yy_te = {"M" : [], "D" : [], "H" : []}
        
    for n in ["M", "D", "H"] :
        for path in range(1,5) :
            X_temp = XXX[n][path-1]
            y_temp = yyy[n][path-1]
            
            X_tn, X_te, y_tn, y_te = train_test_split(X_temp, y_temp, test_size = test_size, random_state = rs)
            
            XX_tn[n].append(X_tn)
            XX_te[n].append(X_te)
            yy_tn[n].append(y_tn)
            yy_te[n].append(y_te)
                  
    for n in ["M", "D", "H"] :
        XX_tn[n] = pd.concat(XX_tn[n])
        XX_te[n] = pd.concat(XX_te[n])
        yy_tn[n] = pd.concat(yy_tn[n])
        yy_te[n] = pd.concat(yy_te[n])
        
        
    X_tn = pd.concat(XX_tn.values())
    X_te = pd.concat(XX_te.values())
    
    y_tn = pd.concat(yy_tn.values())
    y_te = pd.concat(yy_te.values())
    
    return X_tn, X_te, y_tn, y_te

In [16]:
def get_next_tensor(index_list):
    next_mapping = {'M': 0, 'D': 1, 'H': 2}
    next_index = [next_mapping[idx[0]] for idx in index_list] 
    next_tensor = torch.tensor(next_index)
    one_hot = torch.nn.functional.one_hot(next_tensor, num_classes=3).float()
    return one_hot

In [17]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [18]:
def mape(y_true, y_pred, eps=1e-7):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    mask = np.abs(y_true) > eps
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def mape_by_next(y_true, y_pred, next_labels):
    out = {}
    for k in ['M', 'D', 'H']:
        m = (next_labels == k)
        out[k] = mape(y_true[m], y_pred[m])
    return out

In [19]:
results_df = pd.DataFrame(
    columns = ['rs', 'kernel', 'C', 'gamma', 'epsilon',
               'Train MAPE', 'Val MAPE', 'Test MAPE',
               'Next M MAPE', 'Next D MAPE', 'Next H MAPE']
)

random_states = [100, 120, 140, 160, 180]
kernels = ['rbf', 'linear']
Cs = [0.1, 0.5, 1, 5, 10, 100]
gammas = ['scale', 0.01, 0.1]
epsilons = [0.05, 0.1, 0.2]


X, y = Get_Data(dat)


next_map = {'M': 0, 'D': 1, 'H': 2}
def get_next(idx):
    labels = np.array([next_map[i] for i in idx.get_level_values("Next")])
    onehot = np.eye(3, dtype=float)[labels]
    return onehot

for rs, kernel, C, gamma, eps in product(random_states, kernels, Cs, gammas, epsilons):
    setRandomSeed(rs)

    X_tn, X_te, y_tn, y_te = Even_Split(X, y, 1/3, rs)
    X_tr, X_va, y_tr, y_va = Even_Split(X_tn, y_tn, 1/6, rs)

    std_scaler = StandardScaler()
    X_tr_std  = std_scaler.fit_transform(X_tr)
    X_val_std = std_scaler.transform(X_va)
    X_te_std  = std_scaler.transform(X_te)

    next_train = get_next(X_tr.index)
    next_val   = get_next(X_va.index)
    next_test  = get_next(X_te.index)

    X_train = np.concatenate([X_tr_std, next_train], axis=1)
    X_val   = np.concatenate([X_val_std, next_val], axis=1)
    X_test  = np.concatenate([X_te_std, next_test], axis=1)

    y_train_raw = y_tr["Next_SOH"].values.reshape(-1, 1)
    y_val_raw   = y_va["Next_SOH"].values.reshape(-1, 1)
    y_test_raw  = y_te["Next_SOH"].values.reshape(-1, 1)

    y_scaler = StandardScaler()
    y_train = y_scaler.fit_transform(y_train_raw).ravel()
    y_val   = y_scaler.transform(y_val_raw).ravel()
    y_test  = y_scaler.transform(y_test_raw).ravel()


    svr = SVR(kernel=kernel, C=C, gamma=gamma, epsilon=eps, cache_size=2000)

    _ = svr.fit(X_train, y_train)

    y_pred_tr = y_scaler.inverse_transform(svr.predict(X_train).reshape(-1,1)).ravel()
    y_pred_va = y_scaler.inverse_transform(svr.predict(X_val).reshape(-1,1)).ravel()
    y_pred_te = y_scaler.inverse_transform(svr.predict(X_test).reshape(-1,1)).ravel()

    train_mape = mape(y_train_raw.ravel(), y_pred_tr)
    val_mape   = mape(y_val_raw.ravel(), y_pred_va)
    test_mape  = mape(y_test_raw.ravel(), y_pred_te)

    next_test_labels = X_te.index.get_level_values("Next").to_numpy()
    by_next = mape_by_next(y_test_raw.ravel(), y_pred_te, next_test_labels)

    row = [rs, kernel, C, gamma, eps,
           train_mape, val_mape, test_mape,
           by_next['M'], by_next['D'], by_next['H']]

    results_df = pd.concat(
        [results_df, pd.DataFrame([row], columns=results_df.columns)],
        ignore_index=True
    )

In [20]:
summary_df = results_df.groupby(['kernel', 'C', 'gamma', 'epsilon'])[['Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']].mean().reset_index()
summary_df
summary_df.to_csv('SVM_next_info_sum.csv')
results_df.to_csv('SVM_next_info.csv')

,kernel,C,gamma,epsilon,Train MAPE,Val MAPE,Test MAPE,Next M MAPE,Next D MAPE,Next H MAPE
0,linear,0.1,0.01,0.05,0.870006,0.936550,0.901582,0.988119,0.850485,0.866141
1,linear,0.1,0.01,0.10,0.872071,0.919987,0.892844,0.982779,0.832121,0.863633
2,linear,0.1,0.01,0.20,0.875712,0.924835,0.883563,0.967460,0.820899,0.862331
3,linear,0.1,0.1,0.05,0.870006,0.936550,0.901582,0.988119,0.850485,0.866141
4,linear,0.1,0.1,0.10,0.872071,0.919987,0.892844,0.982779,0.832121,0.863633
...,...,...,...,...,...,...,...,...,...,...
103,rbf,100.0,0.1,0.10,0.563965,0.899674,0.874080,1.052752,0.784287,0.785200
104,rbf,100.0,0.1,0.20,0.609320,0.887896,0.858180,1.071691,0.750283,0.752566
105,rbf,100.0,scale,0.05,0.426754,0.998388,0.977472,1.173657,0.902818,0.855940
106,rbf,100.0,scale,0.10,0.461322,0.977306,0.960489,1.145758,0.897678,0.838032
